# Baseline model with prefabricated data



In [0]:
!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Open our new sheet and read some data.
worksheet = gc.open('Namnlöst kalkylark').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df=pd.DataFrame.from_records(rows)
df.head(n=4)

[['Header ', 'Ingress', 'Text', 'Label'], ['Kommunerna förlorare när staten drar sig tillbaka', 'Får betala priset för haveriet med Arbetsförmedlingen', 'Ni kan historien vid det här laget.\n\nDen moderat-kristdemokratiska servettbudgeten – möjliggjord genom Sverigedemokraternas stöd – skar bort 3,6 miljarder från de arbetsmarknadspolitiska åtgärderna. Sedan kom januariavtalet där Centerpartiet fått igenom en privatisering av Arbetsförmedlingen. Myndigheten ska "reformeras i grunden" och den statliga tjänsten ska ersättas av fristående aktörer.\n\nDals-Ed och Bräcke\nArbetsförmedlingen svarade med att varsla 4 500 anställda och meddela att man stänger 132 av sina 242 arbetsförmedlingskontor. Sedan dess har arbetsmarknadsminister Ylva Johansson, Arbetsförmedlingens generaldirektör Mikael Sjöberg och Centerpartiets arbetsmarknadspolitiska talesperson Martin Ådahl bråkat om vems fel slakten är.\n\nFör ute i verkligheten är det ingen som jublar. Oavsett vad man tycker om hur Arbetsförmedli

,0,1,2,3
0,Header,Ingress,Text,Label
1,Kommunerna förlorare när staten drar sig tillbaka,Får betala priset för haveriet med Arbetsförme...,Ni kan historien vid det här laget.\n\nDen mod...,Daniel Swedin
2,Kristerssons reformer betyder mindre välfärd,Oseriöst att kräva sänkta skatter,Den här mandatperiodens viktigaste politiska s...,Daniel Swedin
3,I spansk politik finns bara ytterkanter kvar,Spanskt val kommer inte reda ut kaoset,Under det gångna decenniet har Spanien framstå...,Daniel Swedin


In [0]:
df.columns = df.iloc[0]
df=df.drop([0])
df.head(n=5)

,Header,Ingress,Text,Label
1,Kommunerna förlorare när staten drar sig tillbaka,Får betala priset för haveriet med Arbetsförme...,Ni kan historien vid det här laget.\n\nDen mod...,Daniel Swedin
2,Kristerssons reformer betyder mindre välfärd,Oseriöst att kräva sänkta skatter,Den här mandatperiodens viktigaste politiska s...,Daniel Swedin
3,I spansk politik finns bara ytterkanter kvar,Spanskt val kommer inte reda ut kaoset,Under det gångna decenniet har Spanien framstå...,Daniel Swedin
4,I spansk politik finns bara ytterkanter kvar,Spanskt val kommer inte reda ut kaoset,Under det gångna decenniet har Spanien framstå...,Daniel Swedin
5,Alex & Sigges lyxliv verkar vara känsligt,Klasshatet har fått en klimatdimension,När Malin Persson Giolito skulle skriva sin fj...,Jonna Sima


In [0]:
import numpy as np
import matplotlib.pyplot as plt


# emulate your ex3.Severity.value_counts()
df.Label.value_counts()

Jonna Sima         5
Daniel Swedin      4
Anders Lindberg    4
Name: Label, dtype: int64

In [0]:
import base64
import numpy as np
import pandas as pd

# Plotly imports
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# Other imports
from collections import Counter
from scipy.misc import imread
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from matplotlib import pyplot as plt

In [0]:
pd.value_counts(df['Label']).plot.bar()


<Figure size 432x288 with 1 Axes>


In [0]:
all_words = df['Text'].str.split(expand=True).unstack().value_counts()


In [0]:
x = all_words.index.values[2:50]
y = all_words.values[2:50]

In [0]:
x

array(['i', 'är', 'som', 'har', 'en', 'på', 'det', 'av', 'med', 'för',
       'inte', 'den', 'om', 'ett', 'till', 'de', '–', 'Det', 'Men', 'man',
       'I', 'ska', 'sig', 'men', 'sin', 'mot', 'var', 'vi', 'så', 'från',
       'eller', 'där', 'år', 'han', 'efter', 'De', 'hon', 'bara', 'kan',
       'kommer', 'jag', 'över', 'Sverige', 'blir', 'allt', 'ser', 'vill',
       'upp'], dtype=object)

In [0]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = x
y_pos = np.arange(len(objects))
performance = y
plt.figure(figsize=(10,6))

plt.bar(y_pos, performance, align='center', alpha=0.5,width=0.9)
plt.xticks(y_pos, objects,rotation='vertical')
plt.ylabel('Usage')
plt.title('Programming language usage')
plt.show()

<Figure size 1000x600 with 1 Axes>


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
# Storing the first text element as a string
first_text = df.Text.values[0]
print(first_text)
print("="*90)
print(first_text.split(" "))

Ni kan historien vid det här laget.

Den moderat-kristdemokratiska servettbudgeten – möjliggjord genom Sverigedemokraternas stöd – skar bort 3,6 miljarder från de arbetsmarknadspolitiska åtgärderna. Sedan kom januariavtalet där Centerpartiet fått igenom en privatisering av Arbetsförmedlingen. Myndigheten ska "reformeras i grunden" och den statliga tjänsten ska ersättas av fristående aktörer.

Dals-Ed och Bräcke
Arbetsförmedlingen svarade med att varsla 4 500 anställda och meddela att man stänger 132 av sina 242 arbetsförmedlingskontor. Sedan dess har arbetsmarknadsminister Ylva Johansson, Arbetsförmedlingens generaldirektör Mikael Sjöberg och Centerpartiets arbetsmarknadspolitiska talesperson Martin Ådahl bråkat om vems fel slakten är.

För ute i verkligheten är det ingen som jublar. Oavsett vad man tycker om hur Arbetsförmedlingen sköter sitt uppdrag är myndighetens närvaro viktig för många svenska kommuner, men ännu mer för arbetssökande och företagare.

Om två år så ska så det nya a

In [0]:
first_text_list = nltk.word_tokenize(first_text)
print(first_text_list)

['Ni', 'kan', 'historien', 'vid', 'det', 'här', 'laget', '.', 'Den', 'moderat-kristdemokratiska', 'servettbudgeten', '–', 'möjliggjord', 'genom', 'Sverigedemokraternas', 'stöd', '–', 'skar', 'bort', '3,6', 'miljarder', 'från', 'de', 'arbetsmarknadspolitiska', 'åtgärderna', '.', 'Sedan', 'kom', 'januariavtalet', 'där', 'Centerpartiet', 'fått', 'igenom', 'en', 'privatisering', 'av', 'Arbetsförmedlingen', '.', 'Myndigheten', 'ska', '``', 'reformeras', 'i', 'grunden', "''", 'och', 'den', 'statliga', 'tjänsten', 'ska', 'ersättas', 'av', 'fristående', 'aktörer', '.', 'Dals-Ed', 'och', 'Bräcke', 'Arbetsförmedlingen', 'svarade', 'med', 'att', 'varsla', '4', '500', 'anställda', 'och', 'meddela', 'att', 'man', 'stänger', '132', 'av', 'sina', '242', 'arbetsförmedlingskontor', '.', 'Sedan', 'dess', 'har', 'arbetsmarknadsminister', 'Ylva', 'Johansson', ',', 'Arbetsförmedlingens', 'generaldirektör', 'Mikael', 'Sjöberg', 'och', 'Centerpartiets', 'arbetsmarknadspolitiska', 'talesperson', 'Martin', 'Åd

In [0]:
stopwords_swedish = nltk.corpus.stopwords.words('swedish')
len(stopwords_swedish)

114

In [0]:
stopwords_swedish

['och',
 'det',
 'att',
 'i',
 'en',
 'jag',
 'hon',
 'som',
 'han',
 'på',
 'den',
 'med',
 'var',
 'sig',
 'för',
 'så',
 'till',
 'är',
 'men',
 'ett',
 'om',
 'hade',
 'de',
 'av',
 'icke',
 'mig',
 'du',
 'henne',
 'då',
 'sin',
 'nu',
 'har',
 'inte',
 'hans',
 'honom',
 'skulle',
 'hennes',
 'där',
 'min',
 'man',
 'ej',
 'vid',
 'kunde',
 'något',
 'från',
 'ut',
 'när',
 'efter',
 'upp',
 'vi',
 'dem',
 'vara',
 'vad',
 'över',
 'än',
 'dig',
 'kan',
 'sina',
 'här',
 'ha',
 'mot',
 'alla',
 'under',
 'någon',
 'eller',
 'allt',
 'mycket',
 'sedan',
 'ju',
 'denna',
 'själv',
 'detta',
 'åt',
 'utan',
 'varit',
 'hur',
 'ingen',
 'mitt',
 'ni',
 'bli',
 'blev',
 'oss',
 'din',
 'dessa',
 'några',
 'deras',
 'blir',
 'mina',
 'samma',
 'vilken',
 'er',
 'sådan',
 'vår',
 'blivit',
 'dess',
 'inom',
 'mellan',
 'sådant',
 'varför',
 'varje',
 'vilka',
 'ditt',
 'vem',
 'vilket',
 'sitta',
 'sådana',
 'vart',
 'dina',
 'vars',
 'vårt',
 'våra',
 'ert',
 'era',
 'vilkas']

In [0]:
first_text_list_cleaned = [word for word in first_text_list if word.lower() not in stopwords_swedish]
print(first_text_list_cleaned)
print("="*90)
print("Length of original list: {0} words\n"
      "Length of list after stopwords removal: {1} words"
      .format(len(first_text_list), len(first_text_list_cleaned)))

['historien', 'laget', '.', 'moderat-kristdemokratiska', 'servettbudgeten', '–', 'möjliggjord', 'genom', 'Sverigedemokraternas', 'stöd', '–', 'skar', 'bort', '3,6', 'miljarder', 'arbetsmarknadspolitiska', 'åtgärderna', '.', 'kom', 'januariavtalet', 'Centerpartiet', 'fått', 'igenom', 'privatisering', 'Arbetsförmedlingen', '.', 'Myndigheten', 'ska', '``', 'reformeras', 'grunden', "''", 'statliga', 'tjänsten', 'ska', 'ersättas', 'fristående', 'aktörer', '.', 'Dals-Ed', 'Bräcke', 'Arbetsförmedlingen', 'svarade', 'varsla', '4', '500', 'anställda', 'meddela', 'stänger', '132', '242', 'arbetsförmedlingskontor', '.', 'arbetsmarknadsminister', 'Ylva', 'Johansson', ',', 'Arbetsförmedlingens', 'generaldirektör', 'Mikael', 'Sjöberg', 'Centerpartiets', 'arbetsmarknadspolitiska', 'talesperson', 'Martin', 'Ådahl', 'bråkat', 'vems', 'fel', 'slakten', '.', 'ute', 'verkligheten', 'jublar', '.', 'Oavsett', 'tycker', 'Arbetsförmedlingen', 'sköter', 'sitt', 'uppdrag', 'myndighetens', 'närvaro', 'viktig', '

In [0]:
# Defining our sentence
sentence = ["I love to eat Burgers", 
            "I love to eat Fries"]
vectorizer = CountVectorizer(min_df=0)
sentence_transform = vectorizer.fit_transform(sentence)

In [0]:
sentence_transform

<2x5 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [0]:
sentence_transform[0]

<1x5 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [0]:
print("The features are:\n {}".format(vectorizer.get_feature_names()))
print("\nThe vectorized array looks like:\n {}".format(sentence_transform.toarray()))

The features are:
 ['burgers', 'eat', 'fries', 'love', 'to']

The vectorized array looks like:
 [[1 1 0 1 1]
 [0 1 1 1 1]]


In [0]:
# Define helper function to print top words
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)

In [0]:
text = list(df.Text.values)


In [0]:
tf_vectorizer = CountVectorizer(max_df=0.95, 
                                     min_df=2,
                                     stop_words=stopwords_swedish,
                                     decode_error='ignore')
tf = tf_vectorizer.fit_transform(text)

In [0]:
feature_names = tf_vectorizer.get_feature_names()
count_vec = np.asarray(tf.sum(axis=0)).ravel()
zipped = list(zip(feature_names, count_vec))

In [0]:
feature_names = tf_vectorizer.get_feature_names()
count_vec = np.asarray(tf.sum(axis=0)).ravel()
zipped = list(zip(feature_names, count_vec))
x, y = (list(x) for x in zip(*sorted(zipped, key=lambda x: x[1], reverse=True)))
# Now I want to extract out on the top 15 and bottom 15 words
Y = np.concatenate([y[0:15], y[-16:-1]])
X = np.concatenate([x[0:15], x[-16:-1]])

In [0]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = x
y_pos = np.arange(len(objects))
performance = y
plt.figure(figsize=(80,6))

plt.bar(y_pos, performance, align='center', alpha=0.5,width=0.9)
plt.xticks(y_pos, objects,rotation='vertical')
plt.ylabel('Usage')
plt.title('Programming language usage')
plt.show()

<Figure size 8000x600 with 1 Axes>


## Model 

In [0]:
df.Text

1     Ni kan historien vid det här laget.\n\nDen mod...
2     Den här mandatperiodens viktigaste politiska s...
3     Under det gångna decenniet har Spanien framstå...
4     Under det gångna decenniet har Spanien framstå...
5     När Malin Persson Giolito skulle skriva sin fj...
6     På vägen till jobbet i Stockholm citys hetsiga...
7     Till sist verkar även moderaten Erik Bengtzboe...
8     Hon flydde från Irak till Sverige som sexåring...
9     I fem år har Kristina Winberg och Peter Lundgr...
10    Första maj är Socialdemokraternas hemliga supe...
11    Ulf Kristersson verkar hela valrörelsen och hö...
12    I Peking är det inte ovanligt att man får skan...
13    Tro det eller ej, men i dag är det precis en m...
Name: Text, dtype: object

In [0]:
# https://www.linkedin.com/pulse/text-classification-using-bag-words-approach-nltk-scikit-rajendran

In [0]:
dataset=df
from IPython import embed

In [0]:
df.head(n=3)
df.Text.tolist()

['Ni kan historien vid det här laget.\n\nDen moderat-kristdemokratiska servettbudgeten – möjliggjord genom Sverigedemokraternas stöd – skar bort 3,6 miljarder från de arbetsmarknadspolitiska åtgärderna. Sedan kom januariavtalet där Centerpartiet fått igenom en privatisering av Arbetsförmedlingen. Myndigheten ska "reformeras i grunden" och den statliga tjänsten ska ersättas av fristående aktörer.\n\nDals-Ed och Bräcke\nArbetsförmedlingen svarade med att varsla 4 500 anställda och meddela att man stänger 132 av sina 242 arbetsförmedlingskontor. Sedan dess har arbetsmarknadsminister Ylva Johansson, Arbetsförmedlingens generaldirektör Mikael Sjöberg och Centerpartiets arbetsmarknadspolitiska talesperson Martin Ådahl bråkat om vems fel slakten är.\n\nFör ute i verkligheten är det ingen som jublar. Oavsett vad man tycker om hur Arbetsförmedlingen sköter sitt uppdrag är myndighetens närvaro viktig för många svenska kommuner, men ännu mer för arbetssökande och företagare.\n\nOm två år så ska s

In [0]:
data

['får betala priset haveriet arbetsförmedlingen',
 'oseriöst kräva sänkta skatter',
 'spanskt val kommer reda kaoset',
 'spanskt val kommer reda kaoset',
 'klasshatet fått klimatdimension',
 'företagen väntas expandera sommaren',
 'förnedrande avslut ung politikerkarriär',
 'maria alias vill se kraftsamling hederskulturen',
 'lovar sak röstar tvärtemot europaparlamentet',
 'tillsammans förändra världen',
 'konservativa kannibalismen gör bara m sd kd skyfflar runt väljare moderaterna behöva städa',
 '',
 '']

In [0]:
# creating the feature matrix 
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=1000)
X = matrix.fit_transform(df.Text.tolist()).toarray()
y = df.Label.tolist()


# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# predict class
y_pred = classifier.predict(X_test)

y

['Daniel Swedin',
 'Daniel Swedin',
 'Daniel Swedin',
 'Daniel Swedin',
 'Jonna Sima',
 'Jonna Sima',
 'Jonna Sima',
 'Jonna Sima',
 'Jonna Sima',
 'Anders Lindberg',
 'Anders Lindberg',
 'Anders Lindberg',
 'Anders Lindberg']

In [0]:
print(y_pred)
print(y_test)

['Jonna Sima' 'Daniel Swedin' 'Jonna Sima' 'Jonna Sima']
['Jonna Sima', 'Daniel Swedin', 'Anders Lindberg', 'Anders Lindberg']


In [0]:


# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [0]:
accuracy

0.5